Q 1.1:
Among the linear models, I choose Linear Regression because Cook’s Distance derives specifically from linear regression models. Ridge and Lasso alter coefficient estimates and leverage structure, making influence diagnostics less interpretable.
I use the Statsmodels OLS library because it provides built-in influence diagnostics including Cook’s Distance; Scikit-learn models has special features but are not necessary so far.

In [4]:
#Q 1.2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

import pandas as pd
import statsmodels.api as sm
df = pd.read_csv("usina_with_outliers.csv")

X = df[["AT", "V", "AP", "RH"]]
y = df["PE"]

X_const = sm.add_constant(X)
model = sm.OLS(y, X_const).fit()

#compute cook's distance
influence = model.get_influence()
cooks_output = influence.cooks_distance
cooks_distance_values = cooks_output[0]
df["cooks_distance"] = cooks_distance_values

#defining threshold
n = len(df)
threshold = 4 / n
outliers = df["cooks_distance"] > threshold

df_clean = df.loc[~outliers].drop(columns=["cooks_distance"])
df_clean.to_csv("usina.csv", index=False)


Q3.1
I choose Linear Regression (OLS) using the Statsmodels library because OLS are linear and unbiased. Statsmodels provides a comprehensive summary table—including p-values, standard errors, and t-statistics—necessary for measuring statistical significance. I choose to scale the independent variables because features like AP and AT have very different numeric ranges. By standardizing them, we are able to make all variables comparable. This ensures that "reliability" is measured by how much each feature affects the model per standard deviation change, rather than just reflecting the size of their raw units.

In [6]:
#Q 3.2
df_clean = pd.read_csv("usina.csv")

X = df_clean[["AT", "V", "AP", "RH"]]
y = df_clean["PE"]

scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

X_const = sm.add_constant(X_scaled)

#Fit the OLS model for inference
inference_model = sm.OLS(y, X_const).fit()

#Report the results to identify reliability
print(inference_model.summary())

                            OLS Regression Results                            
Dep. Variable:                     PE   R-squared:                       0.931
Model:                            OLS   Adj. R-squared:                  0.931
Method:                 Least Squares   F-statistic:                 3.188e+04
Date:                Thu, 29 Jan 2026   Prob (F-statistic):               0.00
Time:                        23:48:27   Log-Likelihood:                -27563.
No. Observations:                9448   AIC:                         5.514e+04
Df Residuals:                    9443   BIC:                         5.517e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        454.2997      0.046   9865.884      0.0